<a href="https://colab.research.google.com/github/a00572092/Actividad-7---Regresi-n-Logistica/blob/main/Actividad7_A00572092.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [226]:
#Importar librerías
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

In [227]:
df = pd.read_csv('cuentas_credicel.csv', encoding='ISO-8859-1')
df

<ipython-input-227-db81fc32da0f>:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cuentas_credicel.csv', encoding='ISO-8859-1')


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,HDNAWQ3A,30,12/11/21 19:00,ZTE,Blade A3 2020 RO B,26S,1949.0,780.0,0.0,...,40.02,5000,26,NaN,23.0,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,M418YNR4,33,12/11/21 19:15,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,2999.0,1050.0,0.0,...,35.01,6000,13,NaN,60.0,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,SZ7V3NZT,37,13/11/21 14:13,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,0,74,NaN,58.0,TANTOYUCA,VER.,0,0,0
3,6,9I08UIQZ,38,13/11/21 15:45,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,7000,13,NaN,42.0,AMXTLAN PUE,NaN,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,JMF4LKYU,44,13/11/21 16:10,MOTOROLA,G10 B,39S,4299.0,1075.0,0.0,...,25.01,7000,74,NaN,33.0,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,92KQVRIX,96502,18/04/23 18:45,Motorola,Moto E13,26S,3349.0,297.0,0.0,...,8.87,10000,0,97880,22.0,OXKUTZCAB,YUC,Oxkutzcab,Yucatan,PACT990804MYNCCH05
22731,22968,NWB2EA8J,96479,18/04/23 18:46,Motorola,Moto E22i,26S,4100.0,1209.0,0.0,...,29.49,5000,0,37295,27.0,LEON,GTO,Leon,Guanajuato,HEGB950620MGTRMN03
22732,22969,VDTRJ2W6,96442,18/04/23 18:47,Samsung,A13,26S,4849.0,1152.0,0.0,...,23.76,4750,0,20210,36.0,AGUASCALIENTES,AGS,Aguascalientes,Aguascalientes,GADF870205HASLZL07
22733,22970,PETLF56C,96536,18/04/23 18:49,Samsung,A53 5G,39S,9799.0,2799.0,0.0,...,28.56,7000,0,78434,53.0,SOLEDAD DE GRACIANO SANCHEZ,SLP,Soledad De Graciano Sanchez,San Luis Potosi,DELJ700417HSPLRS03


In [228]:
coeficientes = pd.DataFrame(columns=['Precisión', 'Exactitud', 'Sensibilidad', 'F1'])

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   folio               22735 non-null  int64  
 1   tag                 22735 non-null  object 
 2   folio_solicitud     22735 non-null  int64  
 3   fecha               22735 non-null  object 
 4   marca               22735 non-null  object 
 5   modelo              22735 non-null  object 
 6   plazo               22735 non-null  object 
 7   precio              22735 non-null  float64
 8   enganche            22735 non-null  float64
 9   descuento           22735 non-null  float64
 10  semana              22735 non-null  int64  
 11  monto_financiado    22735 non-null  float64
 12  costo_total         22735 non-null  int64  
 13  monto_accesorios    22735 non-null  float64
 14  agente_venta        22735 non-null  object 
 15  dis_venta           22735 non-null  object 
 16  stat

In [230]:
df.isnull().sum()

folio                    0
tag                      0
folio_solicitud          0
fecha                    0
marca                    0
modelo                   0
plazo                    0
precio                   0
enganche                 0
descuento                0
semana                   0
monto_financiado         0
costo_total              0
monto_accesorios         0
agente_venta             0
dis_venta                0
status                   0
fraude                   0
empresa                 13
inversion                0
pagos_realizados         0
reautorizacion           0
fecha_ultimo_pago     3356
fecha_pago_proximo    3356
status_cuenta         3314
puntos                   0
riesgo                   0
score_buro               0
razones_buro          7340
porc_eng                 0
limite_credito           0
semana_actual            0
cp_cliente            7243
edad_cliente             9
cd_cliente               0
edo_cliente              8
cd_venta                 0
e

#Valores nulos

In [231]:
columna = ['edad_cliente']
df[columna] = df[columna].fillna(round(df[columna].mean(),0))

In [232]:
columna = ['fecha_ultimo_pago', 'fecha_pago_proximo']
df[columna] = df[columna].fillna('01/01/1900 00:00')

In [233]:
df['limite_credito'] = pd.to_numeric(df['limite_credito'], errors='coerce')
df['limite_credito'].fillna(7000, inplace=True)

In [234]:
columna = ['status_cuenta']
df[columna] = df[columna].fillna('Corriente')

In [235]:
columna = ['razones_buro', 'edo_cliente', 'cp_cliente']
df[columna] = df[columna].fillna(method = 'bfill')

In [236]:
columna = ['empresa']
df[columna] = df[columna].fillna(method = 'ffill')

In [237]:
df.isnull().sum()

folio                 0
tag                   0
folio_solicitud       0
fecha                 0
marca                 0
modelo                0
plazo                 0
precio                0
enganche              0
descuento             0
semana                0
monto_financiado      0
costo_total           0
monto_accesorios      0
agente_venta          0
dis_venta             0
status                0
fraude                0
empresa               0
inversion             0
pagos_realizados      0
reautorizacion        0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
puntos                0
riesgo                0
score_buro            0
razones_buro          0
porc_eng              0
limite_credito        0
semana_actual         0
cp_cliente            0
edad_cliente          0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
dtype: int64

In [238]:
df

,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,HDNAWQ3A,30,12/11/21 19:00,ZTE,Blade A3 2020 RO B,26S,1949.0,780.0,0.0,...,40.02,5000.0,26,72490,23.0,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,M418YNR4,33,12/11/21 19:15,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,2999.0,1050.0,0.0,...,35.01,6000.0,13,72490,60.0,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,SZ7V3NZT,37,13/11/21 14:13,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,0.0,74,72490,58.0,TANTOYUCA,VER.,0,0,0
3,6,9I08UIQZ,38,13/11/21 15:45,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,7000.0,13,72490,42.0,AMXTLAN PUE,VER.,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,JMF4LKYU,44,13/11/21 16:10,MOTOROLA,G10 B,39S,4299.0,1075.0,0.0,...,25.01,7000.0,74,72490,33.0,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,92KQVRIX,96502,18/04/23 18:45,Motorola,Moto E13,26S,3349.0,297.0,0.0,...,8.87,10000.0,0,97880,22.0,OXKUTZCAB,YUC,Oxkutzcab,Yucatan,PACT990804MYNCCH05
22731,22968,NWB2EA8J,96479,18/04/23 18:46,Motorola,Moto E22i,26S,4100.0,1209.0,0.0,...,29.49,5000.0,0,37295,27.0,LEON,GTO,Leon,Guanajuato,HEGB950620MGTRMN03
22732,22969,VDTRJ2W6,96442,18/04/23 18:47,Samsung,A13,26S,4849.0,1152.0,0.0,...,23.76,4750.0,0,20210,36.0,AGUASCALIENTES,AGS,Aguascalientes,Aguascalientes,GADF870205HASLZL07
22733,22970,PETLF56C,96536,18/04/23 18:49,Samsung,A53 5G,39S,9799.0,2799.0,0.0,...,28.56,7000.0,0,78434,53.0,SOLEDAD DE GRACIANO SANCHEZ,SLP,Soledad De Graciano Sanchez,San Luis Potosi,DELJ700417HSPLRS03


In [239]:
numero = df.select_dtypes(include=['number'])
categorico = df.select_dtypes(include=['object'])

#Outliers

In [240]:
def remove_outliers_iqr(df, column_name, threshold=1.5):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    return df

for column in numero.columns:
    df = remove_outliers_iqr(df, column)

res = pd.concat([df, categorico], axis=1)
res.shape

(22735, 57)

In [241]:
df= pd.get_dummies(df, columns=['status_cuenta'], drop_first=True)
df= pd.get_dummies(df, columns=['plazo'], drop_first=False)
df = pd.get_dummies(df, columns=['status'], drop_first=False)
df

,folio,tag,folio_solicitud,fecha,marca,modelo,precio,enganche,descuento,semana,...,status_cuenta_Cancelado,status_cuenta_Corriente,status_cuenta_Mora,plazo_13S,plazo_26S,plazo_39S,plazo_52S,status_1,status_2,status_3
7243,7380,RU2XJKH3,27286,19/08/22 14:59,ZTE,BLADE A3 2020 Rojo,2122.82,539.0,0.0,110,...,0,0,0,0,1,0,0,1,0,0
7247,7384,6MCSNDF8,27318,19/08/22 16:35,Samsung,A13,5149.00,882.0,200.0,295,...,0,1,0,0,1,0,0,1,0,0
7250,7387,JJ6G3YBG,27352,19/08/22 17:31,NOKIA,C20 2+32GB AZUL B,3040.83,847.0,0.0,253,...,0,0,1,1,0,0,0,1,0,0
7254,7391,T9BANZI8,27375,19/08/22 19:11,Motorola,XT2167-1 MOTO G41,4799.00,921.0,0.0,268,...,0,0,1,0,1,0,0,1,0,0
7255,7392,7WRXAJDS,27385,19/08/22 19:16,Samsung,A13,5149.00,893.0,200.0,295,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22721,22958,O02NOS0D,96477,18/04/23 17:04,Samsung,A04E,3149.00,810.0,200.0,162,...,0,1,0,0,1,0,0,1,0,0
22722,22959,F43OIVKM,96463,18/04/23 17:05,MOTOROLA,MOTO G31,4749.00,541.0,0.0,232,...,0,1,0,0,0,1,0,1,0,0
22727,22964,BVYCEU9N,96400,18/04/23 17:32,Motorola,Moto G51 5G,5300.00,1325.0,0.0,275,...,0,1,0,0,1,0,0,1,0,0
22731,22968,NWB2EA8J,96479,18/04/23 18:46,Motorola,Moto E22i,4100.00,1209.0,0.0,200,...,0,1,0,0,1,0,0,1,0,0


#Caso 1
Status cuenta Cancelado - Limite credito, porc_eng, score buro

In [242]:
df1=df.copy()
df1['status_cuenta_Cancelado'] = df1['status_cuenta_Cancelado'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df1[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df1['status_cuenta_Cancelado']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [243]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [244]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1888    0]
 [   1    0]]


In [245]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[0] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.9994706193753309

Precisión del modelo:
0.49973530968766544

Puntaje F1 del modelo:
0.4998676198040773



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Caso 2
Status cuenta Corriente - Limite credito, porc_eng, score buro

In [246]:
df2=df.copy()
df2['status_cuenta_Corriente'] = df2['status_cuenta_Corriente'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df2[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df2['status_cuenta_Corriente']
X= Vars_Indep
y= Var_Dep

#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [247]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['yes', 'yes', 'yes', ..., 'yes', 'yes', 'yes'], dtype=object)

In [248]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  611]
 [   0 1278]]


In [249]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[1] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.6765484383271573

Precisión del modelo:
0.33827421916357864

Puntaje F1 del modelo:
0.40353646984527947



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Caso 3
Status cuenta Mora - Limite credito, porc_eng, score buro

In [250]:
df3=df.copy()
df3['status_cuenta_Mora'] = df3['status_cuenta_Mora'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df3[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df3['status_cuenta_Mora']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [251]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [252]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1579    0]
 [ 310    0]]


In [253]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[2] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.8358920063525676

Precisión del modelo:
0.4179460031762838

Puntaje F1 del modelo:
0.45530565167243364



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Caso 4
Plazo 26S - Limite credito, porc_eng, score buro

In [254]:
df4=df.copy()
df4['plazo_26S'] = df4['plazo_26S'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df4[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df4['plazo_26S']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [255]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'yes', ..., 'no', 'yes', 'no'], dtype=object)

In [256]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[286 597]
 [296 710]]


In [257]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[2] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5148306086470982

Exactitud del modelo:
0.5272631021704606

Precisión del modelo:
0.5173188514396443

Puntaje F1 del modelo:
0.5021825001586226



#Caso 5
Plazo 39S - Limite credito, porc_eng, score buro

In [258]:
df5=df.copy()
df5['plazo_39S'] = df5['plazo_39S'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df5[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df5['plazo_39S']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [259]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [260]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1446    0]
 [ 443    0]]


In [261]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[4] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]


Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.7654843832715723

Precisión del modelo:
0.38274219163578616

Puntaje F1 del modelo:
0.4335832083958021



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Caso 6
Plazo 52S - Limite credito, porc_eng, score buro

In [262]:
df6=df.copy()
df6['plazo_52S'] = df6['plazo_52S'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df6[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df6['plazo_52S']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [263]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [264]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1888    0]
 [   1    0]]


In [265]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[5] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.9994706193753309

Precisión del modelo:
0.49973530968766544

Puntaje F1 del modelo:
0.4998676198040773



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Caso 7
Plazo 13S - Limite credito, porc_eng, score buro

In [266]:
df7=df.copy()
df7['plazo_13S'] = df7['plazo_13S'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df7[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df7['plazo_13S']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [267]:
# Encontrar valores únicos en 'Columna1'
valores_unicos_columna1 = df7['fraude'].unique()
print("Valores únicos en fraude:", valores_unicos_columna1)


Valores únicos en fraude: [0]


In [268]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [269]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1441    0]
 [ 446    2]]


In [270]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[6] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5022321428571429

Exactitud del modelo:
0.7638962413975648

Precisión del modelo:
0.8818229994700583

Puntaje F1 del modelo:
0.43743723290598285



#Caso 8
Status3 - Limite credito, porc_eng, score buro

In [271]:
df8=df.copy()
df8['status_3'] = df8['status_3'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df8[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df8['status_3']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [272]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [273]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1887    0]
 [   2    0]]


In [274]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[7] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.9989412387506618

Precisión del modelo:
0.4994706193753309

Puntaje F1 del modelo:
0.4997351694915254



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Caso 9
Status2 - Limite credito, porc_eng, score buro

In [275]:
df9=df.copy()
df9['status_2'] = df9['status_2'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df9[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df9['status_2']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [276]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

In [277]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1662    0]
 [ 227    0]]


In [278]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[8] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.8798305982001059

Precisión del modelo:
0.43991529910005295

Puntaje F1 del modelo:
0.4680371726274289



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Caso 10
Status 1 - Limite credito, porc_eng, score buro

In [279]:
df10=df.copy()
df10['status_1'] = df10['status_1'].replace({1: 'yes', 0: 'no'})
Vars_Indep= df10[['limite_credito', 'porc_eng', 'score_buro']]
Var_Dep= df10['status_1']
X= Vars_Indep
y= Var_Dep
#Split entrenamiento - prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)
#Escalamiento
escalar = StandardScaler()
#Escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
#Algoritmo
algoritmo = LogisticRegression()

In [280]:
#Entrenamiento el modelo
algoritmo.fit(X_train, y_train)
y_pred = algoritmo.predict(X_test)
y_pred

array(['yes', 'yes', 'yes', ..., 'yes', 'yes', 'yes'], dtype=object)

In [281]:
#Matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  224]
 [   0 1665]]


In [282]:
#Coeficientes
sensibilidad = recall_score(y_test, y_pred, average="macro")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()
precision = precision_score(y_test, y_pred, average="macro")
print('Precisión del modelo:')
print(precision)
print()
puntajef1 = f1_score(y_test, y_pred, average="macro")
print('Puntaje F1 del modelo:')
print(puntajef1)
print()
coeficientes.loc[9] = [round(precision,2),round(exactitud,2), round(sensibilidad,2), round(puntajef1,2)]

Sensibilidad del modelo:
0.5

Exactitud del modelo:
0.8814187400741132

Precisión del modelo:
0.4407093700370566

Puntaje F1 del modelo:
0.4684862127180642



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Tabla de coeficientes

In [283]:
coeficientes

,Precisión,Exactitud,Sensibilidad,F1
0,0.50,1.00,0.50,0.50
1,0.34,0.68,0.50,0.40
2,0.52,0.53,0.51,0.50
4,0.38,0.77,0.50,0.43
5,0.50,1.00,0.50,0.50
6,0.88,0.76,0.50,0.44
7,0.50,1.00,0.50,0.50
8,0.44,0.88,0.50,0.47
9,0.44,0.88,0.50,0.47
